In [15]:
import dotenv

dotenv.load_dotenv()

True

In [16]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI

tools = [TavilySearchResults(max_results=1)]

chat = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)

In [17]:
from langchain_community.tools.tavily_search import TavilySearchResults

tool = TavilySearchResults()

In [18]:
tool.invoke({"query": "support.madkudu.com/hc/en-us"})

[{'url': 'https://support.madkudu.com/hc/en-us/articles/4406384883213-Troubleshooting-Login-issues-on-Madkudu-App',
  'content': 'Role "Sales User" only has access to Madkudu Copilot. To use the Single Sign-On login option, your organization must have set up the Okta SSO integration with MadKudu first. If it was set up by the Okta admin of your organization, please reach out to them to assign you to MadKudu (both in Okta and MadKudu).'},
 {'url': 'https://support.madkudu.com/hc/en-us/categories/360002419272-Still-need-help',
  'content': 'Find the multiple options to access MadKudu support Frequently Asked Questions. Troubleshooting: login issues; Why is this record becoming an MQL? If I update an email address on a Lead or Contact in Salesforce, will MadKudu rescore it? ...'},
 {'url': 'https://support.madkudu.com/hc/en-us/articles/360036535232-Introduction-to-MadKudu',
  'content': 'MadKudu Scoring. With MadKudu, qualify your prospects to send to Sales with AI scoring and directly in

In [19]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./madkudu.txt")
data = loader.load()

In [20]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

In [21]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [36]:
retriever = vectorstore.as_retriever(k=4)

docs = retriever.invoke("madkudu support")

docs

[Document(page_content='MadKudu ingests multiple sources of data to help you qualify your prospects and arm your sellers to engage with them', metadata={'source': './madkudu.txt'}),
 Document(page_content='MadKudu ingests multiple sources of data to help you qualify your prospects and arm your sellers to engage with them', metadata={'source': './madkudu.txt'}),
 Document(page_content='Was this article helpful? 1 out of 1 found this helpful\nHave more questions? Submit a request\nReturn to top\nRecently viewed articles\n\nIntroduction to MadKudu Copilot\nHow Marketing Uses MadKudu\nIntroduction to MadKudu\nRelated articles', metadata={'source': './madkudu.txt'}),
 Document(page_content='Was this article helpful? 1 out of 1 found this helpful\nHave more questions? Submit a request\nReturn to top\nRecently viewed articles\n\nIntroduction to MadKudu Copilot\nHow Marketing Uses MadKudu\nIntroduction to MadKudu\nRelated articles', metadata={'source': './madkudu.txt'})]

In [30]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant working at Madkudu. You will need to chat and respond about question to help the user. respond only about madkudu questions.",
        ),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [24]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(chat, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [35]:
from langchain_core.messages import HumanMessage


### This is the main entry point for the agent
agent_executor.invoke({"messages": [HumanMessage(content="what is madkudu")]})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'what is Madkudu'}`


[{'url': 'https://www.madkudu.com/', 'content': "MadKudu reduces the noise for our team, and allows sales and marketing to focus on personalizing outreach to the right accounts. With MadKudu's predictive scores, no more revenue is falling through the cracks. MadKudu predicts and prioritizes revenue-generating actions. Accelerate revenue and drive ROI with MadKudu's revenue automation ..."}]Madkudu is a platform that reduces noise for sales and marketing teams, allowing them to focus on personalizing outreach to the right accounts. It uses predictive scores to prioritize revenue-generating actions and accelerate revenue. You can find more information on their website: [Madkudu](https://www.madkudu.com/)

> Finished chain.


{'messages': [HumanMessage(content='what is madkudu')],
 'output': 'Madkudu is a platform that reduces noise for sales and marketing teams, allowing them to focus on personalizing outreach to the right accounts. It uses predictive scores to prioritize revenue-generating actions and accelerate revenue. You can find more information on their website: [Madkudu](https://www.madkudu.com/)'}

In [33]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

SYSTEM_TEMPLATE = """
Answer the user's questions based on the below context and any information you have about Madkudu.
If the context doesn't contain any relevant information to the question, don't make something up and just say "Im here to respond to madkudu question only":

<context>
{context}
</context>
"""

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_TEMPLATE,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

WITH Context

In [40]:
from langchain_core.messages import HumanMessage

document_chain.invoke(
    {
        "context": docs,
        "messages": [
            HumanMessage(content="How do I perform GDPR-compliant requests in MadKudu?"),
        ],
    }
)

'Im here to respond to madkudu question only'

WITH NO Context

In [39]:
from langchain_core.messages import HumanMessage

document_chain.invoke(
    {
        "context": [],
        "messages": [
            HumanMessage(content="How do I perform GDPR-compliant requests in MadKudu?"),
        ],
    }
)

"MadKudu provides tools and features to help businesses comply with GDPR regulations. To perform GDPR-compliant requests in MadKudu, you can utilize the platform's data management and privacy features to handle data subject requests, such as access, rectification, erasure, and portability. Additionally, MadKudu offers capabilities for managing consent and preferences, as well as data retention and deletion. For specific guidance on performing GDPR-compliant requests in MadKudu, I recommend reaching out to their support team or consulting their documentation for detailed instructions."